## Visualize pitch class features
  
This notebook shows how to load pre-trained models and to generate the visualizations shown in Figure 5 of the paper.

&copy; Christof Weiss and Geoffroy Peeters, Télécom Paris 2021

In [ ]:
import os
import sys
basepath = os.path.abspath(os.path.dirname(os.path.dirname('.')))
sys.path.append(basepath)
import numpy as np, os, scipy, scipy.spatial, matplotlib.pyplot as plt, IPython.display as ipd
from numba import jit
import librosa
import libfmp.b, libfmp.c3, libfmp.c5
import pandas as pd, pickle, re
from numba import jit
import torch.utils.data
import torch.nn as nn
import libdl.data_preprocessing
from libdl.data_loaders import dataset_context, dataset_context_segm
from libdl.nn_models import basic_cnn_segm_sigmoid, basic_cnn_segm_blank_logsoftmax
from libdl.data_preprocessing import compute_hopsize_cqt, compute_hcqt, compute_efficient_hcqt, compute_annotation_array_nooverlap